<a href="https://colab.research.google.com/github/sabari245/tensorflow-to-tensorrt/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

cifar10 = keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
tf.config.list_physical_devices('GPU')

[]

In [ ]:
train_images.shape

(50000, 32, 32, 3)

In [ ]:
def rgb_to_grayscale(images):
    # Use the weighted sum to convert RGB to grayscale
    return np.dot(images, [0.2989, 0.5870, 0.1140])

train_images_grayscale = rgb_to_grayscale(train_images)
test_images_grayscale = rgb_to_grayscale(test_images)

train_images_grayscale.shape

(50000, 32, 32)

In [ ]:
train_labels[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [ ]:
num_classes = tf.reduce_max(tf.cast(train_labels, tf.int32)) + 1
train_labels_1h = tf.one_hot( tf.cast(train_labels, tf.int32)[:, 0], depth=num_classes)
test_labels_1h = tf.one_hot( tf.cast(test_labels, tf.int32)[:, 0], depth=num_classes)

train_labels_1h[:5]

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [ ]:
model_color = keras.Sequential([
    keras.layers.Conv2D(64, (5, 5), padding='same', input_shape=(32, 32, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(10, activation='softmax')
])

model_color.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    fill_mode='nearest'
)


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_lr=0.00001)

history = model_color.fit(
    datagen.flow(train_images, train_labels_1h, batch_size=32),
    epochs=25,
    validation_data=(test_images, test_labels_1h),
    callbacks=[early_stopping, lr_reduction]
)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 57s 32ms/step - accuracy: 0.4086 - loss: 1.8303 - val_accuracy: 0.5925 - val_loss: 1.1534 - learning_rate: 0.0010
Epoch 2/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 44s 28ms/step - accuracy: 0.6088 - loss: 1.1017 - val_accuracy: 0.6386 - val_loss: 1.0307 - learning_rate: 0.0010
Epoch 3/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 0.6639 - loss: 0.9550 - val_accuracy: 0.6434 - val_loss: 1.0551 - learning_rate: 0.0010
Epoch 4/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 82s 27ms/step - accuracy: 0.6991 - loss: 0.8601 - val_accuracy: 0.6608 - val_loss: 0.9936 - learning_rate: 0.0010
Epoch 5/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 81s 26ms/step - accuracy: 0.7252 - loss: 0.7810 - val_accuracy: 0.6678 - val_loss: 1.0077 - learning_rate: 0.0010
Epoch 6/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 84s 27ms/step - accuracy: 0.7408 - loss: 0.7289 - val_accuracy: 0.6867 - val_loss: 0.9804 - learning_rate: 0.0010
Epoch 7/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 42s 27ms/step - accuracy: 0.7598 

In [ ]:
test_loss, test_accuracy = model_color.evaluate(test_images, test_labels_1h)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8407 - loss: 0.4875
Test Loss: 0.4880237877368927
Test Accuracy: 0.8418999910354614


In [ ]:
model_color.save('/content/drive/MyDrive/One_Piece/model_color_new.h5')

In [ ]:
!pip install -U tf2onnx

In [ ]:
!python -m tf2onnx.convert --saved-model "/content/drive/MyDrive/One_Piece/model_color" --output "/content/drive/MyDrive/One_Piece/model_color.onnx" --opset 16

2024-12-18 08:23:31.846127: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 08:23:31.893157: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 08:23:31.907099: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 08:23:36.747582: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2

# Benchmarking the inference time of the model on tensorflow runtime

In [ ]:
from keras.src.saving import load_model
import tensorflow as tf

model = load_model('/content/drive/MyDrive/One_Piece/model_color_new.keras')

In [ ]:
import time

def benchmark_inference_time(model, x_test, num_runs=100):
    """Measures the average inference time."""

    start_time = time.time()
    for _ in range(num_runs):
         model.predict(x_test[0:1]) # pass the data in batches of one
    end_time = time.time()
    avg_inference_time = (end_time - start_time) / num_runs

    start_time = time.time()
    _ = model.predict(x_test)
    end_time = time.time()
    total_time = (end_time - start_time)
    num_images = x_test.shape[0]
    throughput = num_images/total_time
    print(f"Average Inference Time: {avg_inference_time:.6f} seconds per batch")
    print(f"Throughput : {throughput:.2f} images per second")

In [ ]:
benchmark_inference_time(model, test_images)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 636ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

def evaluate_classification_metrics(y_true, y_pred, labels=None):
    """
    Calculates and prints accuracy, precision, recall, and F1-score
    for each class, as well as average metrics.

    Args:
        y_true: True class labels (1D array).
        y_pred: Predicted class labels (1D array).
        labels: Optional; list of labels for printing a classification report.
            If `None` or not given the default behaviour will be applied
    """

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {accuracy:.4f}")

    # Calculate macro average precision, recall, and F1-score
    macro_precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    macro_recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    print(f"Macro-Average Precision: {macro_precision:.4f}")
    print(f"Macro-Average Recall: {macro_recall:.4f}")
    print(f"Macro-Average F1-Score: {macro_f1:.4f}")

    # Print per-class precision, recall, and F1-score using classification_report
    if labels is not None:
      print("Classification Report:")
      print(classification_report(y_true, y_pred, labels = labels, zero_division=0))
    else:
      print("Classification Report:")
      print(classification_report(y_true, y_pred, zero_division=0))

    # print the confusion matrix for in depth look of the prediction
    conf_mat = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix : ")
    print(conf_mat)

In [ ]:
test_pred_labels = model.predict(test_images)
test_pred_labels = np.argmax(test_pred_labels, axis=1)
test_labels = np.argmax(test_labels_1h, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 33s 105ms/step


In [ ]:
evaluate_classification_metrics(test_labels, test_pred_labels)

Accuracy: 0.8419
Macro-Average Precision: 0.8418
Macro-Average Recall: 0.8419
Macro-Average F1-Score: 0.8400
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.92      0.87      1000
           1       0.89      0.94      0.91      1000
           2       0.84      0.76      0.80      1000
           3       0.74      0.65      0.69      1000
           4       0.84      0.80      0.82      1000
           5       0.82      0.75      0.78      1000
           6       0.79      0.93      0.86      1000
           7       0.87      0.88      0.87      1000
           8       0.95      0.88      0.91      1000
           9       0.87      0.91      0.89      1000

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000

Confusion Matrix : 
[[917  10  16   5   4   2   4   6  15  21]
 [  8 940   0   1   0   1   1   0   6  43]
 [ 